In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np


In [2]:
# Transformaciones
transform = transforms.Compose([
    transforms.Resize((64, 64)),   # más chico que 224x224 para ir rápido
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# Rutas del dataset (ajusta si tu carpeta se llama distinto)
train_data_full = datasets.ImageFolder("dataset/train", transform=transform)
test_data_full  = datasets.ImageFolder("dataset/test", transform=transform)

print("Clases:", train_data_full.classes)  # debería mostrar ['sano', 'neumonia']
# Transformaciones
transform = transforms.Compose([
    transforms.Resize((64, 64)),   # más chico que 224x224 para ir rápido
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# Rutas del dataset (ajusta si tu carpeta se llama distinto)
train_data_full = datasets.ImageFolder("dataset/train", transform=transform)
test_data_full  = datasets.ImageFolder("dataset/test", transform=transform)

print("Clases:", train_data_full.classes)  # debería mostrar ['sano', 'neumonia']


Clases: ['neumonia', 'sano']
Clases: ['neumonia', 'sano']


In [3]:
# Queremos solo 15 imágenes de cada clase para train y test

def reducir_dataset(dataset, n=15):
    indices = []
    targets = np.array(dataset.targets)
    for clase in np.unique(targets):
        clase_idx = np.where(targets == clase)[0][:n]  # primeros n
        indices.extend(clase_idx)
    return Subset(dataset, indices)

train_data = reducir_dataset(train_data_full, 15)
test_data  = reducir_dataset(test_data_full, 15)

# Loaders
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=4, shuffle=False)

print("Tamaño train:", len(train_data))
print("Tamaño test:", len(test_data))


Tamaño train: 30
Tamaño test: 30


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # entrada 3 canales RGB
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 64)  # depende del tamaño de entrada (64x64)
        self.fc2 = nn.Linear(64, 2)  # 2 clases: sano / neumonia

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)  # a vector
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # tasa de aprendizaje


In [6]:
epochs = 5

for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Época {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

print("Entrenamiento finalizado ")


Época 1/5, Loss: 0.7283
Época 2/5, Loss: 0.6812
Época 3/5, Loss: 0.6207
Época 4/5, Loss: 0.4730
Época 5/5, Loss: 0.2894
Entrenamiento finalizado ✅


In [7]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy en test: {accuracy:.2f}%")


Accuracy en test: 90.00%


In [8]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(3*64*64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = x.view(-1, 3*64*64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

mlp = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp.parameters(), lr=0.001)


In [9]:
# Entrenamiento rápido
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = mlp(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"[MLP] Época {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Evaluación
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = mlp(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_mlp = 100 * correct / total
print(f"Accuracy MLP en test: {accuracy_mlp:.2f}%")


[MLP] Época 1/5, Loss: 0.6921
[MLP] Época 2/5, Loss: 0.4068
[MLP] Época 3/5, Loss: 0.3553
[MLP] Época 4/5, Loss: 0.1682
[MLP] Época 5/5, Loss: 0.0553
Accuracy MLP en test: 70.00%


In [10]:
print(f"Accuracy CNN: {accuracy:.2f}%")
print(f"Accuracy MLP: {accuracy_mlp:.2f}%")



Accuracy CNN: 90.00%
Accuracy MLP: 70.00%
